In [2]:
import os

# Define the folder structure
folder_structure = {
    "selenium_pytest_project": {
        "configurations": [],
        "page_objects": [],
        "test_cases": [],
        "test_data": [],
        "utilities": [],
        "reports": []
    }
}

# Function to create folders
def create_folders(base_path, folder_structure):
    for folder, subfolders in folder_structure.items():
        folder_path = os.path.join(base_path, folder)
        
        # Create the main project folder
        if not os.path.exists(folder_path):
            os.makedirs(folder_path)
            print(f"✅ Created folder: {folder_path}")
        
        # Create subfolders inside each main folder
        for subfolder in subfolders:
            subfolder_path = os.path.join(folder_path, subfolder)
            if not os.path.exists(subfolder_path):
                os.makedirs(subfolder_path)
                print(f"✅ Created subfolder: {subfolder_path}")

# Base path where the project folder will be created
base_path = os.getcwd()  # Current working directory

# Create the folder structure
create_folders(base_path, folder_structure)

print("✅ Folder structure created successfully.")


✅ Folder structure created successfully.


In [5]:
import requests

def get_cognito_access_token(client_id, client_secret, token_url):
    # Prepare the data for the token request
    data = {
        'grant_type': 'client_credentials',
    }

    # Make the POST request to get the token
    response = requests.post(token_url, data=data, auth=(client_id, client_secret))

    # Check if the request was successful
    if response.status_code == 200:
        # Parse the JSON response
        token_info = response.json()
        return token_info['access_token']
    else:
        # Handle error
        raise Exception(f"Failed to get token: {response.status_code} - {response.text}")

# Replace these with your Cognito App Client ID and Secret
client_id = '1m2f2ib1qatacreojll4aommfo'
client_secret = 'o30f697g1s7fm9jrrlveeiu4tok3aulpie90709g86gijvccc2n'

# Replace this with your Cognito token endpoint (Stg or Prod)
token_url = 'https://logiai-stg.auth.us-west-2.amazoncognito.com/oauth2/token'

access_token = get_cognito_access_token(client_id, client_secret, token_url)


Extraction the required html elements

In [1]:
import os
from bs4 import BeautifulSoup

# Paths
input_html_path = 'raw_profiles.html'  # Original HTML
output_folder = 'chunks_for_profiles'  # Folder to save the chunks

# Function to create folder if it doesn't exist
def create_folder(folder_path):
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
        print(f"✅ Folder '{folder_path}' created.")
    else:
        print(f"✅ Folder '{folder_path}' already exists.")

# Function to extract meaningful testing elements and split into chunks
def extract_and_chunk_elements(input_html_path, output_folder, chunk_size=50):
    with open(input_html_path, 'r', encoding='utf-8') as file:
        html_content = file.read()

    # Parse the full HTML
    soup = BeautifulSoup(html_content, 'html.parser')

    # Create a new BeautifulSoup object for the cleaned output
    new_soup = BeautifulSoup("<html><body></body></html>", 'html.parser')

    # Elements important for Selenium Testing
    important_tags = []

    # --- Form Controls ---
    important_tags += soup.find_all(['input', 'button', 'select', 'textarea'])

    # --- Links that can navigate ---
    for link in soup.find_all('a', href=True):
        if link['href'] != '#' and not link['href'].startswith('javascript'):
            important_tags.append(link)

    # --- Headings ---
    important_tags += soup.find_all(['h1', 'h2', 'h3', 'h4', 'h5', 'h6'])

    # --- List items ---
    important_tags += soup.find_all('li')

    # --- Optional: Divs or spans that contain visible text or action ---
    for tag in soup.find_all(['div', 'span']):
        if tag.get_text(strip=True):  # Only if they have visible text
            important_tags.append(tag)

    # Create the output folder if it doesn't exist
    create_folder(output_folder)

    # Split into chunks
    total_elements = len(important_tags)
    chunk_count = (total_elements // chunk_size) + (1 if total_elements % chunk_size else 0)

    for i in range(chunk_count):
        chunk_start = i * chunk_size
        chunk_end = chunk_start + chunk_size

        # Create a new soup object for the chunk
        chunk_soup = BeautifulSoup("<html><body></body></html>", 'html.parser')

        # Append a chunk of elements to the body
        for tag in important_tags[chunk_start:chunk_end]:
            chunk_soup.body.append(tag)

        # File path for the chunk
        chunk_file_path = os.path.join(output_folder, f"beta-marketPlace_for_testing_chunk_{i+1}.html")

        # Write the chunk to a new HTML file
        with open(chunk_file_path, 'w', encoding='utf-8') as output_file:
            output_file.write(chunk_soup.prettify())
        
        print(f"✅ Chunk {i+1} saved to: {chunk_file_path}")

# Run the extraction and chunking
extract_and_chunk_elements(input_html_path, output_folder, chunk_size=50)


✅ Folder 'chunks_for_profiles' created.
✅ Chunk 1 saved to: chunks_for_profiles\beta-marketPlace_for_testing_chunk_1.html
✅ Chunk 2 saved to: chunks_for_profiles\beta-marketPlace_for_testing_chunk_2.html
✅ Chunk 3 saved to: chunks_for_profiles\beta-marketPlace_for_testing_chunk_3.html
✅ Chunk 4 saved to: chunks_for_profiles\beta-marketPlace_for_testing_chunk_4.html
✅ Chunk 5 saved to: chunks_for_profiles\beta-marketPlace_for_testing_chunk_5.html


In [8]:
import openai
import os

# Function to extract code from OpenAI response
def extract_code(response_text):
    code_blocks = response_text.split("```python")
    if len(code_blocks) > 1:
        return code_blocks[1].split("```")[0].strip()
    return response_text.strip()

# Function to read all chunk files from a folder
def read_chunks_from_folder(chunks_folder):
    chunk_files = []
    for filename in os.listdir(chunks_folder):
        file_path = os.path.join(chunks_folder, filename)
        if filename.endswith(".html") or filename.endswith(".txt"):
            with open(file_path, 'r', encoding='utf-8') as file:
                chunk_content = file.read()
                chunk_files.append((filename, chunk_content))
    return chunk_files

# Function to generate Page Object files
def generate_pom_from_chunks(project_folder, chunks_folder):
    # Define page_objects folder path
    page_objects_folder = os.path.join(project_folder, "page_objects")
    os.makedirs(page_objects_folder, exist_ok=True)

    # Read all chunks
    chunks = read_chunks_from_folder(chunks_folder)
    if not chunks:
        print(f"❌ No valid HTML chunk files found in {chunks_folder}")
        return

    # OpenAI client setup
    client = openai.OpenAI(
        base_url="https://stg-logiai-service.np.logitech.io/openai/v1",
        api_key=access_token  # Make sure this is already available
    )

    for filename, chunk_content in chunks:
        # Create a simple prompt for each chunk
        prompt = f"""
        I am creating a Selenium Page Object Model (POM) for a webpage.

        Below is an HTML snippet:
        {chunk_content}

        Based on this HTML, please generate a Python Page Object Class:
        - Class name should be based on the file name (without extension), in PascalCase.
        - Use Selenium locators and best practices.
        - Create `__init__` method and basic element locators.
        - No test cases — only page elements and actions if obvious.
        - Use `By` from Selenium.
        - Return the code inside a Python code block.
        """

        try:
            completion = client.chat.completions.create(
                model="gpt-4",
                messages=[{"role": "user", "content": prompt}],
                stream=False,
                max_tokens=1200,
            )

            # Extract the code
            pom_code = extract_code(completion.choices[0].message.content)

            # Convert filename to class name
            base_name = os.path.splitext(filename)[0]
            class_name = "".join(word.capitalize() for word in base_name.split("_"))

            # Save file
            pom_file_path = os.path.join(page_objects_folder, f"{base_name}.py")
            with open(pom_file_path, 'w', encoding='utf-8') as file:
                file.write(pom_code)

            print(f"✅ Created Page Object: {pom_file_path}")

        except Exception as e:
            print(f"❌ Error processing {filename}: {e}")

# === Main Run ===
# Input paths
waste_folder = r"C:\genai\ai_for_automation\wastage\pom_chunks_profiles"
chunks_folder = r"C:\genai\ai_for_automation\chunks_for_profiles"

# Run function
generate_pom_from_chunks(waste_folder, chunks_folder)


✅ Created Page Object: C:\genai\ai_for_automation\wastage\pom_chunks_profiles\page_objects\beta-marketPlace_for_testing_chunk_1.py
✅ Created Page Object: C:\genai\ai_for_automation\wastage\pom_chunks_profiles\page_objects\beta-marketPlace_for_testing_chunk_2.py
✅ Created Page Object: C:\genai\ai_for_automation\wastage\pom_chunks_profiles\page_objects\beta-marketPlace_for_testing_chunk_3.py
✅ Created Page Object: C:\genai\ai_for_automation\wastage\pom_chunks_profiles\page_objects\beta-marketPlace_for_testing_chunk_4.py
✅ Created Page Object: C:\genai\ai_for_automation\wastage\pom_chunks_profiles\page_objects\beta-marketPlace_for_testing_chunk_5.py


In [9]:
import openai
import os

def read_all_chunks_from_folder(folder_path):
    """Reads and combines all .py files from the given folder."""
    combined_content = ""
    for filename in sorted(os.listdir(folder_path)):
        if filename.endswith(".py"):
            with open(os.path.join(folder_path, filename), 'r', encoding='utf-8') as f:
                combined_content += f.read() + "\n\n"
    return combined_content

def extract_code(response_text):
    """Extracts Python code from markdown-style response.""" 
    if "```python" in response_text:
        return response_text.split("```python")[1].split("```")[0].strip()
    return response_text.strip()

def send_to_llm(file_content, api_key, base_url):
    """Sends the combined content to the LLM and returns optimized code."""
    client = openai.OpenAI(
        base_url=base_url,
        api_key=api_key
    )

    prompt = (
        "You are a Python Selenium expert following the Page Object Model (POM).\n"
        "Below is a class split across chunks. Please:\n"
        "- Combine all chunks into one clean class.\n"
        "- Keep ALL functions (do not remove any).\n"
        "- Do NOT change any locators.\n"
        "- Remove duplicate function names.\n"
        "- Ensure class is clean and well-structured.\n"
        "- go through every function if there is any logic mistake try to write a comment after the function. \n"
        "- try to create all the function in one single class"
        f"{file_content}"
    )

    try:
        completion = client.chat.completions.create(
            model="gpt-4",
            messages=[{"role": "user", "content": prompt}],
            stream=False,
            max_tokens=9000,
        )
        return extract_code(completion.choices[0].message.content.strip())
    except Exception as e:
        print(f"Error from LLM: {e}")
        return ""

def create_output_file(output_file_path, optimized_content):
    """Writes optimized code to the specified output file."""
    try:
        with open(output_file_path, 'w', encoding='utf-8') as output_file:
            output_file.write(optimized_content)
        print(f" Optimized file created at {output_file_path}")
    except Exception as e:
        print(f"Error writing to file: {e}")

def process_folder(folder_path, output_file_path, api_key, base_url):
    """Reads all chunk files, combines them, sends to LLM, and writes result."""
    file_content = read_all_chunks_from_folder(folder_path)
    if not file_content:
        print(f"No Python files found in: {folder_path}")
        return

    optimized_content = send_to_llm(file_content, api_key, base_url)
    if not optimized_content:
        print("No optimized content returned.")
        return

    create_output_file(output_file_path, optimized_content)

# Example usage
input_folder = r"C:\genai\ai_for_automation\wastage\pom_chunks_profiles\page_objects"
output_file = r"C:\genai\ai_for_automation\selenium_pytest_project\page_objects\page_objects_profile.py"
api_key = access_token  # Replace or define this
base_url = "https://stg-logiai-service.np.logitech.io/openai/v1"

process_folder(input_folder, output_file, api_key, base_url)


 Optimized file created at C:\genai\ai_for_automation\selenium_pytest_project\page_objects\page_objects_profile.py


In [51]:
import openai
import os

# === Utility Functions ===

def read_files_from_folder(folder_path):
    contents = []
    try:
        for filename in sorted(os.listdir(folder_path)):
            file_path = os.path.join(folder_path, filename)
            if os.path.isfile(file_path):
                with open(file_path, 'r', encoding='utf-8') as file:
                    contents.append(file.read())
        return contents
    except Exception as e:
        print(f"Error reading files from folder: {e}")
        return []

def read_file(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            return f.read()
    except Exception as e:
        print(f"Error reading file {file_path}: {e}")
        return ""

def extract_code(response_text):
    if "```python" in response_text:
        return response_text.split("```python")[1].split("```")[0].strip()
    return response_text.strip()

def send_to_llm_single_chunk(html_chunk, pom_code, conftest_code, api_key, base_url):
    from openai import OpenAI
    client = OpenAI(api_key=api_key, base_url=base_url)

    prompt = (
        "You are an expert in Selenium and Pytest.\n"
        "Generate a Python test function using pytest and the provided conftest fixture and Page Object Model (POM).\n"
        "Use correct imports and create only testcases and import the the page object.\n"
        "user setup method for launching a driver once and start testing "
        f"HTML Snippet:\n{html_chunk}\n\n"
        f"POM Code:\n{pom_code}\n\n"
        f"conftest.py:\n{conftest_code}"
    )

    try:
        completion = client.chat.completions.create(
            model="gpt-4o",
            messages=[{"role": "user", "content": prompt}],
            max_tokens=4096
        )
        return extract_code(completion.choices[0].message.content.strip())
    except Exception as e:
        print(f"LLM call failed: {e}")
        return ""



def save_combined_test(output_file_path, test_code):
    try:
        with open(output_file_path, 'w', encoding='utf-8') as f:
            f.write(test_code)
        print(f"Final test file saved at: {output_file_path}")
    except Exception as e:
        print(f"Error saving final test file: {e}")

# === Main Function ===

def generate_single_test_file(html_folder, pom_file_path, conftest_path, output_file_path, api_key, base_url):
    html_chunks = read_files_from_folder(html_folder)
    pom_code = read_file(pom_file_path)
    conftest_code = read_file(conftest_path)

    combined_test_code = ""

    for i, chunk in enumerate(html_chunks):
        print(f"\n--- Processing chunk {i+1}/{len(html_chunks)} ---")
        test_code = send_to_llm_single_chunk(chunk, pom_code, conftest_code, api_key, base_url)
        if test_code:
            combined_test_code += f"# === Test from chunk {i+1} ===\n{test_code}\n\n"
        else:
            print(f"No test code received for chunk {i+1}.")

    if combined_test_code:
        save_combined_test(output_file_path, combined_test_code)
    else:
        print("No test code generated.")


# === Update these with correct paths ===

html_folder = r"C:\genai\ai_for_automation\chunks_for_profiles"
pom_file_path = r"C:\genai\ai_for_automation\selenium_pytest_project\page_objects\page_objects_profile.py"  # Use correct single POM file
conftest_path = r"C:\genai\ai_for_automation\selenium_pytest_project\configurations\configurations\conftest.py"
output_file_path = r"C:\genai\ai_for_automation\selenium_pytest_project\test_cases\test_final_profiles.py"
api_key = access_token
base_url = "https://stg-logiai-service.np.logitech.io/openai/v1"

generate_single_test_file(html_folder, pom_file_path, conftest_path, output_file_path, api_key, base_url)



--- Processing chunk 1/5 ---


INFO:httpx:HTTP Request: POST https://stg-logiai-service.np.logitech.io/openai/v1/chat/completions "HTTP/1.1 200 OK"



--- Processing chunk 2/5 ---


INFO:httpx:HTTP Request: POST https://stg-logiai-service.np.logitech.io/openai/v1/chat/completions "HTTP/1.1 200 OK"



--- Processing chunk 3/5 ---


...


--- Processing chunk 4/5 ---


INFO:httpx:HTTP Request: POST https://stg-logiai-service.np.logitech.io/openai/v1/chat/completions "HTTP/1.1 200 OK"



--- Processing chunk 5/5 ---


INFO:httpx:HTTP Request: POST https://stg-logiai-service.np.logitech.io/openai/v1/chat/completions "HTTP/1.1 200 OK"


Final test file saved at: C:\genai\ai_for_automation\selenium_pytest_project\test_cases\test_final_profiles.py


In [52]:
import openai
import os
import logging

# Setup logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

def read_single_file(file_path):
    """Reads and returns the content of a single file."""
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            return file.read()
    except Exception as e:
        logger.error(f"Error reading file {file_path}: {e}")
        return ""

def extract_code(response_text):
    """Extracts Python code from markdown-style response.""" 
    if "```python" in response_text:
        return response_text.split("```python")[1].split("```")[0].strip()
    return response_text.strip()

def send_to_llm(file_content, api_key, base_url):
    """Sends a single file's content to the LLM and returns optimized Python code."""
    client = openai.OpenAI(
        base_url=base_url,
        api_key=api_key
    )

    prompt = f"""
You are a Selenium expert using Pytest.

You will be given a merged raw test case file composed of multiple test chunks. Your task is to:

1. Consolidate the code into a **single clean test file**.
2. **Do NOT remove or rename any test functions** — all test logic must be preserved.
3. **Eliminate duplicate imports** and organize them properly.
4. Remove **duplicate test cases** (same name and logic).
5. Ensure that **each test function is unique**, well-structured, and properly formatted.
6. Clean and organize fixtures and helper methods without changing their behavior.

IMPORTANT:
- **Preserve test logic and data** from all chunks.
- Ensure **all test cases and fixtures are retained** unless they are exact duplicates.
- Adjust **import paths consistently** (e.g., unify `from POM` vs `from pom` vs `from plugin_marketplace_page`).
- Ensure **no syntax errors** and that the file is ready to run with `pytest`.

Return a single, clean, final test file with all necessary test functions and imports.

Here is the raw test file:

{file_content}
"""
    try:
        completion = client.chat.completions.create(
            model="gpt-4",
            messages=[{"role": "user", "content": prompt}],
            stream=False,
            max_tokens=9000,
        )
        return extract_code(completion.choices[0].message.content.strip())
    except Exception as e:
        logger.error(f"Error while sending content to LLM: {e}")
        return ""

def create_output_file(output_file_path, optimized_content):
    """Writes optimized code to output file."""
    try:
        with open(output_file_path, 'w', encoding='utf-8') as output_file:
            output_file.write(optimized_content)
        logger.info(f" Optimized file created at {output_file_path}")
    except Exception as e:
        logger.error(f"Error writing to output file: {e}")

def process_single_file(input_file_path, output_file_path, api_key, base_url):
    """Main function for processing a single file."""
    file_content = read_single_file(input_file_path)
    if not file_content:
        logger.warning(f"No content found in file: {input_file_path}")
        return

    optimized_content = send_to_llm(file_content, api_key, base_url)
    if not optimized_content:
        logger.warning("No optimized content received from LLM.")
        return

    create_output_file(output_file_path, optimized_content)

# Example usage
input_file_path = r"C:\genai\ai_for_automation\selenium_pytest_project\test_cases\test_final_profiles.py"
output_file_path = r"C:\genai\ai_for_automation\selenium_pytest_project\test_cases\test_case_profiles.py"
api_key = access_token  # Make sure access_token is defined or imported
base_url = "https://stg-logiai-service.np.logitech.io/openai/v1"

process_single_file(input_file_path, output_file_path, api_key, base_url)


INFO:httpx:HTTP Request: POST https://stg-logiai-service.np.logitech.io/openai/v1/chat/completions "HTTP/1.1 200 OK"
INFO:__main__: Optimized file created at C:\genai\ai_for_automation\selenium_pytest_project\test_cases\test_case_profiles.py


In [48]:
import openai
import os
import logging

# ============ Logging Configuration ============
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# ============ Utility Functions ============

def read_single_file(file_path):
    """
    Reads the content of a single file and returns the content as a string.
    """
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            return file.read()
    except Exception as e:
        logger.error(f"Error reading file {file_path}: {e}")
        return ""

def read_html_from_folder(folder_path):
    """
    Reads all HTML files from a specified folder and combines their contents into a single string.
    """
    combined_html = []
    try:
        for file_name in os.listdir(folder_path):
            file_path = os.path.join(folder_path, file_name)
            if file_name.endswith(".html"):  # Only process HTML files
                html_chunk = read_single_file(file_path)
                if html_chunk:
                    combined_html.append(html_chunk)
                    logger.info(f"Read HTML chunk from file: {file_name}")
        return "\n".join(combined_html)  # Combine all chunks into a single string
    except Exception as e:
        logger.error(f"Error reading HTML files from folder {folder_path}: {e}")
        return ""

def truncate_file_content(file_content, max_length=6000):
    """
    Truncates file content to a specified length to avoid token limit issues.
    Appends a comment indicating truncation.
    """
    if len(file_content) > max_length:
        logger.warning(f"Input file content exceeds {max_length} characters. Truncating...")
        return file_content[:max_length] + "\n# TRUNCATED CONTENT DUE TO TOKEN LIMIT"
    return file_content

def extract_code(response_text):
    """
    Extracts Python code blocks from a model's markdown response.
    If no `python` code block is found, returns the raw response text.
    """
    try:
        if "```python" in response_text:
            return response_text.split("```python")[1].split("```")[0].strip()
    except Exception as e:
        logger.error(f"Error extracting Python code: {e}")
    return response_text.strip()

def create_output_file(output_file_path, optimized_content):
    """
    Writes the provided content to a file at the specified path.
    """
    try:
        with open(output_file_path, 'w', encoding='utf-8') as output_file:
            output_file.write(optimized_content)
        logger.info(f"Optimized file successfully created at {output_file_path}")
    except Exception as e:
        logger.error(f"Error writing to file {output_file_path}: {e}")


# ============ Core Functionality ============
def send_to_llm(file_content, api_key, base_url, html_chunks, pom_code="", conftest_code=""):
    """
    Sends the file content to the language model (LLM) for processing
    and returns a Python code file optimized for Selenium and Pytest.
    """
    # Set OpenAI API configurations
    openai.api_key = api_key
    openai.api_base = base_url

    # Construct the prompt
    prompt = f"""
    You are an expert in Selenium and Pytest.

    Using **Page Object Model (POM)**, generate Python test cases for testing a web application's functionality. 
    Focus on the following:

    1. **Automation Context:** Write test cases to validate user interactions, navigation, buttons, links, or input fields.
    2. Use the given **POM class and conftest fixture** to structure the test cases.
    3. Consolidate the code from the given raw test file into a **single consolidated Python test file**.
    4. Optimize execution by ensuring the browser launches **only once using a session/module-level fixture**. 
    5. Create output with:
        - Well-named test functions, meaningful docstrings, and proper assertions.
        - Reuse of test setup fixtures for all test cases.
        - Readable, clean, and modular Python design.

    Inputs:
    - **Combined HTML Snippets**: \n{html_chunks}
    - **Page Object Model (POM) Code**: \n{pom_code}
    - **conftest.py Code**: \n{conftest_code}

    Here is the raw test file for consolidation and optimization:\n{file_content}
    """

    try:
        # Send request to OpenAI API
        response = openai.ChatCompletion.create(
            model="gpt-4",
            messages=[{"role": "user", "content": prompt}],
            max_tokens=3000,  # Adjust token limit based on needs
            temperature=0  # For deterministic results
        )
        # Extract and return optimized code
        return extract_code(response.choices[0].message["content"])
    except Exception as e:
        logger.error(f"Error while sending content to LLM: {e}")
        return ""


def process_single_file(input_file_path, output_file_path, html_folder_path, api_key, base_url, pom_code, conftest_code):
    """
    Main function to process a single test case file with the LLM.
    Reads the input file, sends it to the LLM for optimization, and writes the output.
    Combines HTML files from a folder into the prompt.
    """
    file_content = read_single_file(input_file_path)
    if not file_content:
        logger.warning(f"No content found in file: {input_file_path}")
        return

    # Read all HTML chunks from the folder
    html_chunks = read_html_from_folder(html_folder_path)
    if not html_chunks:
        logger.warning(f"No HTML chunks found in folder: {html_folder_path}")
        return

    # Truncate content if necessary
    truncated_content = truncate_file_content(file_content)

    # Send the content to LLM
    optimized_content = send_to_llm(truncated_content, api_key, base_url, html_chunks, pom_code, conftest_code)
    if not optimized_content:
        logger.warning("No optimized content received from LLM.")
        return

    # Create an output file with the optimized code
    create_output_file(output_file_path, optimized_content)


# ============ Example Usage ============
if __name__ == "__main__":
    # Define file paths
    input_file_path = r"C:\genai\ai_for_automation\selenium_pytest_project\test_cases\test_final_profiles.py"
    output_file_path = r"C:\genai\ai_for_automation\selenium_pytest_project\test_cases\test_case_profiles.py"
    html_folder_path = r"C:\genai\ai_for_automation\chunks_for_profiles"  # Folder containing HTML files

    # Define API Key and Base URL
    api_key = os.getenv("OPENAI_API_KEY", "your_openai_api_key_here")  # Avoid hardcoding API keys
    base_url = "https://stg-logiai-service.np.logitech.io/openai/v1"  # Replace with actual URL if applicable

    # Define the additional inputs for the prompt
    pom_code = read_single_file(r"C:\genai\ai_for_automation\selenium_pytest_project\page_objects\page_objects_profile.py")  # Adjust path
    conftest_code = read_single_file(r"selenium_pytest_project/configurations/configurations/conftest.py")  # Adjust path

    # Process the file
    process_single_file(input_file_path, output_file_path, html_folder_path, api_key, base_url, pom_code, conftest_code)

INFO:__main__:Read HTML chunk from file: beta-marketPlace_for_testing_chunk_1.html
INFO:__main__:Read HTML chunk from file: beta-marketPlace_for_testing_chunk_2.html
INFO:__main__:Read HTML chunk from file: beta-marketPlace_for_testing_chunk_3.html
INFO:__main__:Read HTML chunk from file: beta-marketPlace_for_testing_chunk_4.html
INFO:__main__:Read HTML chunk from file: beta-marketPlace_for_testing_chunk_5.html
ERROR:__main__:Error while sending content to LLM: 

You tried to access openai.ChatCompletion, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742

